une introduction aux listes d'initialisation

L'initialisation détermine naturellement la valeur initiale d'une variable (jusqu'ici rien de neuf on enfonce tranquillement les portes ouvertes). Avant c ++ 11, il y avait (déjà) trois formes syntaxiques pour exprimer l'initialisation:

```c++
int k = 12;
int l (12);
int j = {12};
```

La première est l'initialisation classique, la seconde est l'initialisation faite comme un appel de fonction (on aime bien celle là) et la troisième ... et bien elle se comportera différemment suivant que le programme est compilé en `c++11` ou dans une version de c++ préalable à c++11 donc ou l'oublie (on en a déjà bien assez).

`c++11` propose une nouvelle forme syntaxique (oui oui une quatrième qui modifie d'ailleurs la troisième) pour exprimer l'initialisation. Elle prend la forme d'une liste `{}` c'est pourquoi on l'appelle la liste d'initialisation mais aussi on va vraiment s'en servir pour initialiser, par exemple des conteneurs, où on a besoin d'une liste d'objets.

```c++
int i {12};
```

Alors ce que se passe-t-il ? Alors dejà, vous pouvez utiliser le `{}-initializer` dans tous les contextes où une initialisation doit être effectuée et il est fortement recommandé (par B.J. Stroustrup) de le faire ! Alors on va écouter Bjarne et comprendre ce que cela nous apporte.

Alors pourquoi c++11 introduit-il une nouvelle forme d'initialisation ?

Et bien cette nouvelle forme va vous prévenir quand, lors de cette initialisation, c++ fait une conversion implicite (implicite parce que vous ne le demandez pas directement) qui peut vous faire perdre de l'information.

Prenons un exemple, je vais chercher, sur mon ordi, le plus grand `int`

```c++
// in file main.cpp
#include <iostream>
#include <limits>

int main () {
  std::cout << std::numeric_limits<int>::max() << std::endl;
  return 0;
}
```

```c++
$ g++ main.cpp -o max
$ ./max
2147483647
```

Maintenant je vais ajouter 1 à cet entier qui va donc dépasser la taille des `int` et essayer d'initialiser une variable de type `int` avec:

```c++
// in file main.cpp
#include <iostream>
int main () {
  int i = 2147483648;
  std::cout << i << std::endl;
  return 0;
    
}
```

```c++
$ g++ main.cpp -o pb
$ ./pb
-2147483648
```

Oui effectivement c'est pas bon du tout tout cela ! La valeur $2147483648$ ne *tient pas* dans un espace de type `int`, quand on essaie d'initialiser une variable de type `int` avec cette valeur, c++ fait une grosse conversion implicite sans rien dire ! ce grand entier est devenu tout petit (si vous avez remarqué le signe négatif et vous remarquez au passage que c'est le plus petit `int`).  

Nous commencons à comprendre ce qu'une conversion implicite est, et nous voyons qu'on peut y perdre beaucoup d'information !

Déjà une première réflexion avant d'accuser le langage c++ de faire n'importe quoi. C'est vous qui écrivez votre code, vous devez maîtriser ce que vous faites. Là, pour l'exemple, j'utilise un très grand nombre 2147483648 et j'initialise une variable de type `int` à partir de ce nombre mais clairement dans un code sérieux il aurait fallu s'inquiéter de l'éventail des valeurs dans lequel évoluent nos nombres et nous aurions vu que nous ne pouvons pas utiliser le type `int` mais que nous devons recourrir à des types permettant de stocker de plus grandes valeurs que les `int` par exemple des `long int` voire des `long long int` voire des bibliothèques extérieures de grands nombres. Ces Undefined Behavior

Mais revenons à notre exemple dans lequel ca se passe plutôt mal: c++ vient de transformer un 2147483648 en un -2147483648 sns rien nous dire ! Alors si il y a une possibilité que c++11 nous prévienne lorsque ce genre de conversion est faite ! il faut l'utiliser.

```c++
// in file main.cpp
#include <iostream>

int main () {
  int i {2147483648};
  std::cout << i << std::endl;
  return 0;
}
```

Nous remarquons le `{}-initializer` autour de l'entier trop grand pour être un `int`. Compilons ce fichier:

```c++ 
g++ main.cpp
main.cpp: In function ‘int main()’:
main.cpp:4:20: error: narrowing conversion of ‘2147483648’ from ‘long int’ to ‘int’ inside { } [-Wnarrowing]
   int i {2147483648};
                    ^
```


Voila ce fichier ne compile plus ! c++ par ce moyen va interdire, sur ses types de base (dits built-in), une conversion lorsqu'elle vous fait perdre de l'information (dite implicit narrowing conversion). Naturellement comme toujours vous pouvez forcer c++ à faire quelque chose qu'il refuse. 

```c++ 
$ g++ main.cpp -Wno-narrowing -o bad
$ ./bad
-2147483648      ^
```

Ici j'utilise l'option de compilation -Wno-narrowing sur mon compilateur dont la version est:
```c++
$ g++ --version
g++ (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
```

Au passage, vous voyez que j'utilise le compilateur `g++` (`g` pour gnu) qui est un logiciel libre de la `Free Software Foundation` de Richard Stallman (allez lire ce que c'est là par exemple https://en.wikipedia.org/wiki/Free_Software_Foundation) et vous voyez qu'il me donne une information de `NO
warranty`. Alors oui effectivement la seule garantie que vous ayez en utilisant ce compilateur c'est qu'il est libre d'utilisatiom donc il est utilisé par des millions de personnes (et de nombreux étudiants) et que le moindre bug ou problème est soigneusement traqué donc même sans garantie on peu lui faire confiance.

De la même manière, vous serez averti si une conversion implicite est faite lorsqu'il y a conversion entre types, par exemple vous essayez de convertir un `float` en `int` et que vous perdez de l'information.

La version sans `{}-initializer`

```c++
// in file main.cpp
#include <iostream>
int main () {
  float f = 3.14;
  int i = f; 
  std::cout << i << std::endl;
  return 0;
}
```

```c++
$ g++ main.cpp
3
```

et bien oui on s'y attendait `i` est égal à la partie entière de 3.14

La version avec `{}-initializer`

```c++
// in file main.cpp
#include <iostream>

int main () {
  float f = 3.14;
  int j {f};
  std::cout << j << std::endl;
  return 0;
}
```

```c++
$ g++ main.cpp
g++ init_list.cpp -Wnarrowing
init_list.cpp: In function ‘int main()’:
init_list.cpp:5:11: warning: narrowing conversion of ‘f’ from ‘float’ to ‘int’ inside { } [-Wnarrowing]
   int j {f};
           ^
$ ./a.out
3
```

Cette fois vous avez été prévenu ! (ici un warning de compilation pas une erreur de compilation, mon exećutable a été généré). 

Compliquons les choses. Je prend mon plus grand `int` et je l'affiche. Ensuite je l'augmente de 1, je l'affecte à un `int` en prenant garde à bien demander une `{}-initializer` et je l'affiche.

```c++
// in file main.cpp
#include <iostream>
#include <limits>


int main () {
  int max = std::numeric_limits<int>::max();
  std::cout << "mon plus grand int " << max << std::endl;
  int i {max+1};
  std::cout << "mon plus grand int +1" << i << std::endl;
  return 0;
}
```


Je le compile et je l'exécute

```c++
$ g++ main.cpp
$ ./a.out
mon plus grand int est 2147483647
mon plus grand int augmenté de 1 est -2147483648
```

Fichtre ! mais il ne me dit rien ... pourtant j'ai bien demander une `{}-initializer` de max+1.

J'essaie autre chose, je fais cette fois un simple affichage de max+1 (plus de `int` et plus de `{}-initializer`):
```c++
// in file main.cpp
int main () {
  int max = std::numeric_limits<int>::max();
  std::cout << "mon plus grand int est " << max << std::endl;
  std::cout << "mon plus grand int augmenté de 1 est " << max+1 << std::endl;
  return 0;
}
```
Je le compile et je l'exécute

```c++
$ g++ main.cpp
$ ./a.out
mon plus grand int est 2147483647
mon plus grand int augmenté de 1 est -2147483648
```

Là aussi ca se passe très mal avec ou sans `{}-initializer` de max+1, la conversion implicite se fait !

Pas de panique, je vous explique. c++ sait que 2147483647 est un `int` (c'est même sur mon ordinateur son plus grand `int`) et 1 est aussi un `int`. Quand je fais 2147483647 + 1, c++ voit qu'il doit réaliser pour cette expression l'addition de deux `int`. Il doit choisir le type du résultat de l'expression. En effet, même si vous ne le voyez pas, le résultat sera naturellement rangé dans une zone mémoire à un moment donné quelque part. Quel est le type que c++ va utiliser pour cette mémoire (donc pour le résultat de l'expression) ? ...  un indice: il calcule l'addition de deux `int` ... donc le plus simple pour lui est d'utiliser une mémoire de type: `int` pour le résultat et donc voila d'où vient cet étrange débordement (overflow) qui fait que le résultat de l'expression max+1 soit toujours -2147483648.

Ce problème existe naturellement aussi lorsqu'on compile du `c`. La norme du `c` nous dit *If an exceptional condition occurs during the evaluation of an expression (that is, if the result is not mathematically defined or not in the range of representable values for its type), the behavior is undefined.*. Ce qui va aussi être le cas lors de divisions par zéro.

Pour une lecture introductive, vous pouvez vous arrêter là, le reste du notebook n'est pas de l'apprentissage de la programmation mais une discussion tout à fait discutable par ailleurs.

Alors quoi faire ?

La chose la plus importante est d'être conscient que ce genre de problème peut se produire.

Ensuite de manière complètement abusive et unilatérale, je vais vous dire d'attendre d'être confronté à un problème pour le traiter: rien de pire que de pré-traiter tous vos programmes pour tous les problèmes possibles imaginables qui pourraient se poser un jour, en n'oubliant aucun scénario catastrophique les pires possibles ... vous allez faire des codes super compliqués et super lents, que le compilateur ne va plus pouvoir optimiser (oui parce qu'en plus il optimise beaucoup et heureusement ... sinon vos programmes ne seraient pas aussi rapides).

Enfin, par exemple, il va exister des options de compilation qui vont mettre des pièges dans un programme (des trap) afin qu'une erreur se déclenche dès que l'exécution du programme fait un tel overflow (dépassement de capacité) lors d'opérations arithmétique d'addition, soustraction et multiplication.... Heu non pas la division par zéro qui va être traitée autrement... oui rien n'est simple mais c++ pointe naturellement les problèmes des langages qui sont proches de l'ordinateur, si vous n'êtes pas obligé de vous soucier de performances pensez à utilisez un langage plus simple à manipuler comme `python 3` où, par exemple, le type `int` n'est plus borné (https://docs.python.org/3/whatsnew/3.0.html#integers) mais vous en payerez le prix en temps et en place mémoire.

Donc, je disais, sur mon compilateur `gnu` c'est l'option `-ftrapv` qui va faire en sorte que l'exécution d'un programme déclenche un `abort` si un tel overflow arrive (le fameux signal abort ou SIGABRT qui est une termination anormale d'un processus).

```c++
// in file main.cpp
#include <iostream>
#include <limits>

int main () {
  int max = std::numeric_limits<int>::max();
  std::cout << max+1 << std::endl;
  return 0;
}
```

je compile avec l'option `-ftrapv` et j'exécute le programme

```c++
g++ -g main.cpp -ftrapv
$ ./a.out
Aborted (core dumped)
```

ahhh super ! le programme se termine vraiment très mal

Et enfin, regardez ce programme minimaliste:

```c++
#include <iostream>
#include <limits>

int main () {
  int max = std::numeric_limits<int>::max();
  max+1;
  return 0;
}
```

je compile avec l'option `-ftrapv` et j'exécute le programme

```c++
g++ -g main.cpp -ftrapv
$ ./a.out
$
```

ben quoi qu'est ce ? que se passe-t-il ? où est mon abort ???

optimisation je vous disais... vous ne pensez tout de même pas qu'un langage comme c++ va traiter du code alors qu'il se rend compte qu'il est inutile ! non bien sûr, votre max+1 n'est simplement pas regardé !

END